In [3]:
# Import all the necessary modules
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas_datareader as pdr
import math
import datetime
import itertools
import yfinance as yf
import seaborn as sn
from IPython.display import display, HTML
from trend_following import (apply_jupyter_fullscreen_css, load_financial_data, get_returns_volatility, calculate_slope, trend_signal, slope_signal, 
                             create_trend_strategy, get_close_prices, calculate_donchian_channels)
from strategy_performance_metrics import (calculate_sharpe_ratio, calculate_calmar_ratio, calculate_CAGR, calculate_risk_and_performance_metrics,
                                          calculate_compounded_cumulative_returns, estimate_fee_per_trade, rolling_sharpe_ratio)
import coinbase_utils as cn
%matplotlib inline

In [4]:
import importlib
importlib.reload(cn)

<module 'coinbase_utils' from '/Users/adheerchauhan/Documents/git/trend_following/coinbase_utils.py'>

In [7]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('Display.max_rows', None)
pd.set_option('Display.max_columns',None)
apply_jupyter_fullscreen_css()

In [9]:
from IPython.display import display, HTML

def jupyter_interactive_mode():
    display(HTML('''
    <style>
        /* Set the notebook container width to full screen */
        .jp-Notebook {
            width: 100% !important;
            max-width: 100% !important;
            margin-left: 0px !important;
            margin-right: 0px !important;
        }
        
        /* Full width for notebook cells, align cells to the left */
        .jp-Cell {
            display: block; /* Change to block to align cells left */
        }
        .jp-Cell .jp-InputArea, .jp-Cell .jp-OutputArea {
            width: 100% !important;
        }
        
        /* Full width for the main output area */
        .jp-OutputArea {
            width: 100% !important;
        }
        
        /* Remove the top padding */
        .jp-NotebookPanel {
            padding-top: 0px !important;
        }
        
        /* Remove extra bottom spacing */
        .jp-Notebook .jp-Cell-end {
            height: 0px !important;
        }
    </style>
    '''))

# Call the function to apply the style
jupyter_interactive_mode()


## Helper functions to help pull the data and run the analysis

## Pull the data from Yahoo Finance

We begin by pulling daily data for the specified instrument using the Yahoo finance API. The data available only goes back to late 2017. 

## Moving Average and Donchian Channels Signals

In [13]:
df = cn.get_coinbase_ohlc_data(ticker='BTC')
# df = (df[['close']].rename(columns={'close':ticker}))
df = df[(df.index.get_level_values('date') >= in_sample_start_date) & (df.index.get_level_values('date') <= in_sample_end_date)]

NameError: name 'in_sample_start_date' is not defined

In [15]:
df.tail()

,low,high,open,close,volume
date,,,,,
2024-09-26,62652.99,65865.00,63131.13,65177.21,12789.850355
2024-09-27,64827.85,66550.00,65176.39,65789.00,9529.016881
2024-09-28,65400.00,66266.39,65787.53,65858.99,2755.226806
2024-09-29,65433.29,66059.00,65857.33,65600.01,2721.351977
2024-09-30,62833.95,65624.95,65600.01,63301.25,10629.741479


In [11]:
from strategy_performance_metrics import calculate_risk_and_performance_metrics

import seaborn as sns

def plot_moving_avg_crossover_performance(df_performance, ticker):
    unique_step_sizes = df_performance['stepsize'].unique()

    # Plotting each heatmap in a loop
    for step in unique_step_sizes:
        subset = df_performance[df_performance['stepsize'] == step]
        pivoted_df_sharpe = subset.pivot(index='slow_mavg', columns='fast_mavg', values='sharpe_ratio')
        pivoted_df_calmar = subset.pivot(index='slow_mavg', columns='fast_mavg', values='calmar_ratio')
        pivoted_df_return = subset.pivot(index='slow_mavg', columns='fast_mavg', values='annualized_return')
        
        fig = plt.figure(figsize=(30,6))
        # plt.style.use('bmh')
        layout = (1,3)
        sharpe_ax = plt.subplot2grid(layout, (0,0))#, colspan=2)
        calmar_ax = plt.subplot2grid(layout, (0,1))#, colspan=2)
        return_ax = plt.subplot2grid(layout, (0,2))#, colspan=2)

        sns.heatmap(pivoted_df_sharpe, annot=True, fmt=".2f", cmap='RdYlGn', linewidths=.5, ax=sharpe_ax)
        _ = sharpe_ax.set_title(f'{ticker} Sharpe Ratio Heatmap\nStep Size: {step}')
        _ = sharpe_ax.set_ylabel('Slow Moving Average (Days)')
        _ = sharpe_ax.set_xlabel('Fast Moving Average (Days)')

        sns.heatmap(pivoted_df_calmar, annot=True, fmt=".2f", cmap='RdYlGn', linewidths=.5, ax=calmar_ax)
        _ = calmar_ax.set_title(f'{ticker} Calmar Ratio Heatmap\nStep Size: {step}')
        _ = calmar_ax.set_ylabel('Slow Moving Average (Days)')
        _ = calmar_ax.set_xlabel('Fast Moving Average (Days)')

        sns.heatmap(pivoted_df_return, annot=True, fmt=".2f", cmap='RdYlGn', linewidths=.5, ax=return_ax)
        _ = return_ax.set_title(f'{ticker} Annualized Return Heatmap\nStep Size: {step}')
        _ = return_ax.set_ylabel('Slow Moving Average (Days)')
        _ = return_ax.set_xlabel('Fast Moving Average (Days)')

        plt.tight_layout()
    
    return

def generate_trend_signal_with_donchian_channel(start_date, end_date, ticker, fast_mavg, slow_mavg, mavg_stepsize, moving_avg_type='exponential', price_or_returns_calc='returns',
                                                rolling_donchian_window=20, include_signal_strength=True, long_only=False):
    
    # Generate Trend Signal
    # df = get_close_prices(start_date, end_date, ticker, print_status=False)
    df = cn.get_coinbase_ohlc_data(ticker='BTC')
    df = (df[['close']].rename(columns={'close':ticker}))
    df = df[(df.index.get_level_values('date') >= start_date) & (df.index.get_level_values('date') <= end_date)]
    df_trend = (create_trend_strategy(df, ticker, mavg_start=fast_mavg, mavg_end=slow_mavg, mavg_stepsize=mavg_stepsize, slope_window=10, moving_avg_type=moving_avg_type,
                                      price_or_returns_calc=price_or_returns_calc)
                .rename(columns={f'{ticker}_trend_strategy_returns': f'{ticker}_trend_strategy_returns_{fast_mavg}_{mavg_stepsize}_{slow_mavg}',
                                 f'{ticker}_trend_strategy_trades': f'{ticker}_trend_strategy_trades_{fast_mavg}_{mavg_stepsize}_{slow_mavg}'}))
    # Generate Donchian Signal
    df_donchian = calculate_donchian_channels(start_date=start_date, end_date=end_date, ticker=ticker, price_or_returns_calc=price_or_returns_calc,
                                            rolling_donchian_window=rolling_donchian_window)
    if price_or_returns_calc == 'price':

        # Buy signal: Price crosses above upper band
        df_donchian[f'{ticker}_{rolling_donchian_window}_donchian_signal'] = np.where(
            (df_donchian[f'close'] > df_donchian[f'{ticker}_{rolling_donchian_window}_donchian_upper_band_price']), 1,
            np.where((df_donchian[f'close'] < df_donchian[f'{ticker}_{rolling_donchian_window}_donchian_lower_band_price']), -1, 0))
    elif price_or_returns_calc == 'returns':
        df_donchian[f'{ticker}_{rolling_donchian_window}_donchian_signal'] = np.where(
            (df_donchian[f'{ticker}_pct_returns'] > df_donchian[f'{ticker}_{rolling_donchian_window}_donchian_upper_band_returns']), 1,
            np.where((df_donchian[f'{ticker}_pct_returns'] < df_donchian[f'{ticker}_{rolling_donchian_window}_donchian_lower_band_returns']), -1, 0))

    # Merging the Trend and donchian Dataframes
    donchian_cols = [f'{ticker}_{rolling_donchian_window}_donchian_upper_band_{price_or_returns_calc}', f'{ticker}_{rolling_donchian_window}_donchian_lower_band_{price_or_returns_calc}',
                     f'{ticker}_{rolling_donchian_window}_donchian_middle_band_{price_or_returns_calc}', f'{ticker}_{rolling_donchian_window}_donchian_signal']
    df_trend = pd.merge(df_trend, df_donchian[donchian_cols], left_index=True, right_index=True, how='left')
    
    if include_signal_strength:
        # Calculate the strength of moving average crossover and donchian signal
        df_trend[f'{ticker}_donchian_band_width_{price_or_returns_calc}'] = (df_trend[f'{ticker}_{rolling_donchian_window}_donchian_upper_band_{price_or_returns_calc}'] -
                                                                            df_trend[f'{ticker}_{rolling_donchian_window}_donchian_lower_band_{price_or_returns_calc}'])
        donchian_strength = (np.abs(df_trend[f'{ticker}'] - df_trend[f'{ticker}_{rolling_donchian_window}_donchian_middle_band_{price_or_returns_calc}']) /
                            df_trend[f'{ticker}_donchian_band_width_{price_or_returns_calc}'])
        crossover_strength = np.abs(df_trend[f'{ticker}_{fast_mavg}_mavg'] - df_trend[f'{ticker}_{slow_mavg}_mavg']) / df_trend[f'{ticker}']

        df_trend[f'{ticker}_crossover_donchian_signal_strength'] = (donchian_strength + crossover_strength) / 2
        strength_threshold = 0.5
    
        # Moving Average and Donchian Channel Signal
        buy_signal = ((df_trend[f'{ticker}_{rolling_donchian_window}_donchian_signal'] == 1) &
                      (df_trend[f'{ticker}_trend_signal'] == 1) &
                      (df_trend[f'{ticker}_crossover_donchian_signal_strength'] > strength_threshold))
        sell_signal = ((df_trend[f'{ticker}_{rolling_donchian_window}_donchian_signal'] == -1) &
                       (df_trend[f'{ticker}_trend_signal'] == -1) &
                       ((df_trend[f'{ticker}_crossover_donchian_signal_strength'] > strength_threshold)))
    else:
        # Moving Average and Donchian Channel Signal
        buy_signal = ((df_trend[f'{ticker}_{rolling_donchian_window}_donchian_signal'] == 1) &
                      (df_trend[f'{ticker}_trend_signal'] == 1))
        sell_signal = ((df_trend[f'{ticker}_{rolling_donchian_window}_donchian_signal'] == -1) &
                       (df_trend[f'{ticker}_trend_signal'] == -1))
    
    if long_only:
        df_trend[f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_signal'] = (
            np.where(buy_signal, 1, 0))
    else:
        df_trend[f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_signal'] = (
            np.where(buy_signal, 1, np.where(sell_signal, -1, 0)))
        
    df_trend[f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'] = (
        df_trend[(f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_signal')] * 
        df_trend[f'{ticker}_pct_returns'])
    df_trend[f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_trades'] = (
        df_trend[(f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_signal')].diff())
    
    return df_trend

def moving_avg_crossover_with_donchian_strategy_performance(start_date, end_date, ticker, moving_avg_type='exponential', price_or_returns_calc='returns',
                                                           rolling_donchian_window=20, include_signal_strength=True, long_only=False):
    
    perf_cols = ['ticker', 'fast_mavg', 'slow_mavg', 'stepsize', 'annualized_return', 'sharpe_ratio', 'calmar_ratio', 'annualized_std_dev', 'max_drawdown', 'max_drawdown_duration',
                'hit_rate', 't_statistic', 'p_value', 'trade_count']
    df_performance = pd.DataFrame(columns=perf_cols)
    
    fast_mavg_list = np.arange(10, 101, 10)
    slow_mavg_list = np.arange(50, 501, 50)
    mavg_stepsize_list = [2, 4, 6, 8]
    performance_rows = []
    for slow_mavg in slow_mavg_list:
        for fast_mavg in fast_mavg_list:
            for mavg_stepsize in mavg_stepsize_list:
                if fast_mavg < slow_mavg:
                    df_trend = generate_trend_signal_with_donchian_channel(start_date, end_date, ticker, fast_mavg, slow_mavg, mavg_stepsize, moving_avg_type, price_or_returns_calc,
                                                                         rolling_donchian_window, include_signal_strength, long_only)
                    performance_metrics = calculate_risk_and_performance_metrics(
                        df_trend, strategy_daily_return_col=f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns',
                        strategy_trade_count_col=f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_trades',
                        annual_trading_days=365, transaction_cost_est=0.005)
                    performance_rows.append({
                        'ticker': ticker,
                        'fast_mavg': fast_mavg,
                        'slow_mavg': slow_mavg,
                        'stepsize': mavg_stepsize,
                        'annualized_return': performance_metrics['annualized_return'],
                        'sharpe_ratio': performance_metrics['annualized_sharpe_ratio'],
                        'calmar_ratio': performance_metrics['calmar_ratio'],
                        'annualized_std_dev': performance_metrics['annualized_std_dev'],
                        'max_drawdown': performance_metrics['max_drawdown'],
                        'max_drawdown_duration': performance_metrics['max_drawdown_duration'],
                        'hit_rate': performance_metrics['hit_rate'],
                        't_statistic': performance_metrics['t_statistic'],
                        'p_value': performance_metrics['p_value'],
                        'trade_count': performance_metrics['trade_count']
                    })
                    # df_performance = df_performance.append(row, ignore_index=True)
    # Convert the list of rows to a DataFrame
    df_performance = pd.DataFrame(performance_rows, columns=perf_cols)
    
    plot_moving_avg_crossover_performance(df_performance, ticker)
    
    return df_performance

In [13]:
def plot_trend_following_performance(df, start_date, end_date, ticker, fast_mavg, slow_mavg, mavg_stepsize, price_or_returns_calc, rolling_donchian_window, rolling_sharpe_window):
    
    start_date = pd.to_datetime(start_date).date().strftime('%Y-%m-%d')
    end_date = pd.to_datetime(end_date).date().strftime('%Y-%m-%d')
    fig = plt.figure(figsize=(22,20))
    layout = (5,2)
    trend_ax = plt.subplot2grid(layout, (0,0), colspan=2)
    trend_donchian_ax = plt.subplot2grid(layout, (1,0), colspan=2)
    trend_signal_ax = plt.subplot2grid(layout, (2,0), colspan=2)
    trend_rolling_sharpe_ax = plt.subplot2grid(layout, (3,0), colspan=2)
    trend_return_ax = plt.subplot2grid(layout, (4,0))#, colspan=2)
    trend_cum_return_ax = plt.subplot2grid(layout, (4,1))#, colspan=2)

    if price_or_returns_calc == 'price':
        _ = trend_ax.plot(df.index, df[f'{ticker}'], label='Price')
    elif price_or_returns_calc == 'returns':
        _ = trend_ax.plot(df.index, df[f'{ticker}_pct_returns'], label='Returns')
    for mavg in np.linspace(fast_mavg, slow_mavg, mavg_stepsize):
        _ = trend_ax.plot(df.index, df[f'{ticker}_{int(mavg)}_mavg'], label=f'{mavg} M Avg')

    _ = trend_ax.set_title(f'{ticker} Moving Average Ribbons - {start_date} - {end_date}')
    _ = trend_ax.set_ylabel('Price')
    _ = trend_ax.set_xlabel('Date')
    _ = trend_ax.legend()

    if price_or_returns_calc == 'price':
        _ = trend_donchian_ax.plot(df.index, df[f'{ticker}'], label='Price')
    elif price_or_returns_calc == 'returns':
        _ = trend_donchian_ax.plot(df.index, df[f'{ticker}_pct_returns'], label='Returns')
    _ = trend_donchian_ax.plot(df.index,
                      df[f'{ticker}_{rolling_donchian_window}_donchian_upper_band_{price_or_returns_calc}'], label='Donchian Upper Band', linestyle='--', linewidth=3)
    _ = trend_donchian_ax.plot(df.index,
                      df[f'{ticker}_{rolling_donchian_window}_donchian_lower_band_{price_or_returns_calc}'], label='Donchian Lower Band', linestyle='--', linewidth=3)

    _ = trend_donchian_ax.set_title(f'{ticker} Donchian Channels')
    _ = trend_donchian_ax.set_ylabel('Price')
    _ = trend_donchian_ax.set_xlabel('Date')
    _ = trend_donchian_ax.legend()

    _ = trend_signal_ax.plot(df.index,
                             df[f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_signal'], label='Signal')
    _ = trend_signal_ax.set_title(f'{ticker} Trend Strategy with Donchian Channel Signal')
    _ = trend_signal_ax.set_ylabel('Signal')
    _ = trend_signal_ax.set_xlabel('Date')
    _ = trend_signal_ax.legend()

    _ = trend_rolling_sharpe_ax.plot(df.index,
                                     df[f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_rolling_sharpe_{rolling_sharpe_window}'],
                                     label='Sharpe Ratio', color='orange')
    _ = trend_rolling_sharpe_ax.set_title(f'{ticker} Rolling Sharpe Ratio')
    _ = trend_rolling_sharpe_ax.set_ylabel('Sharpe Ratio')
    _ = trend_rolling_sharpe_ax.set_xlabel('Date')
    _ = trend_rolling_sharpe_ax.legend()

    _ = trend_return_ax.plot(df.index,
                             df[f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'],
                             label='Return')
    _ = trend_return_ax.set_title(f'{ticker} Trend Strategy with Donchian Channel Return')
    _ = trend_return_ax.set_ylabel('Return')
    _ = trend_return_ax.set_xlabel('Date')
    _ = trend_return_ax.legend()

    # _ = trend_cum_return_ax.plot(df_trend_mavg_donchian.index, df_trend_mavg_donchian[f'{ticker}_trend_strategy_returns_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_cum'], label='Cum. Return')
    _ = trend_cum_return_ax.plot(df.index, df['strategy_cumulative_return'], label='Cum. Return')
    _ = trend_cum_return_ax.set_title(f'{ticker} Trend Strategy with Donchian Channel Cumulative Return')
    _ = trend_cum_return_ax.set_ylabel('Cum. Return')
    _ = trend_cum_return_ax.set_xlabel('Date')
    _ = trend_cum_return_ax.legend()


    plt.tight_layout()
    
    return

def get_trend_following_strategy_performance(start_date, end_date, ticker, fast_mavg, slow_mavg, mavg_stepsize, rolling_donchian_window, rolling_sharpe_window, long_only=False,
                                             include_transaction_costs_and_fees=True, transaction_cost_est=0.001, passive_trade_rate=0.5):
    price_or_returns_calc = 'price'
    df = generate_trend_signal_with_donchian_channel(start_date, end_date, ticker, fast_mavg=fast_mavg, slow_mavg=slow_mavg,
                                                     mavg_stepsize=mavg_stepsize, price_or_returns_calc=price_or_returns_calc, rolling_donchian_window=rolling_donchian_window,
                                                     include_signal_strength=False, long_only=long_only)
    if include_transaction_costs_and_fees:
        average_fee_per_trade = estimate_fee_per_trade(passive_trade_rate=passive_trade_rate)
        adjusted_daily_returns = df[f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'] - (
            np.abs(df[f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_trades']) * (transaction_cost_est + average_fee_per_trade))
        df['strategy_cumulative_return'] = (1 + adjusted_daily_returns).cumprod() - 1
    else:
        df['strategy_cumulative_return'] = (1 + df[f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns']).cumprod() - 1

    df[f'rolling_sharpe_{rolling_sharpe_window}'] = (rolling_sharpe_ratio(df, window=rolling_sharpe_window, 
    strategy_daily_return_col=f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns',
    strategy_trade_count_col=f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_trades'))
    df = df.rename(
        columns={f'rolling_sharpe_{rolling_sharpe_window}': f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_rolling_sharpe_{rolling_sharpe_window}'})
    
    plot_trend_following_performance(df, start_date, end_date, ticker, fast_mavg, slow_mavg, mavg_stepsize, price_or_returns_calc, rolling_donchian_window, rolling_sharpe_window)
    
    return df

In [15]:
## In Sample Performance
in_sample_start_date = pd.Timestamp(2014, 9, 17).date()
in_sample_end_date = pd.Timestamp(2022, 12, 31).date()
out_of_sample_start_date = pd.Timestamp(2023, 1, 1).date()
out_of_sample_end_date = pd.Timestamp(2024, 9, 30).date()
full_sample_start_date = pd.Timestamp(2014, 9, 17).date()
full_sample_end_date = pd.Timestamp(2024, 9, 30).date()

## Long Short Strategy Performance

### In Sample & Out of Sample Performance

In [ ]:
ticker_list = ['BTC-USD','ETH-USD','LTC-USD','SOL-USD','DOGE-USD']
in_sample_performance_dict = {}
out_of_sample_performance_dict = {}
for ticker in ticker_list:
    ## Price with Donchian Channels and without Signal Strength Signal
    df_long_short_strategy_performance_in_sample = moving_avg_crossover_with_donchian_strategy_performance(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker=ticker,
                                                                                                           price_or_returns_calc='price', include_signal_strength=False, long_only=False)
    df_long_short_strategy_performance_out_of_sample = moving_avg_crossover_with_donchian_strategy_performance(start_date=out_of_sample_start_date, end_date=out_of_sample_end_date, ticker=ticker,
                                                                                                               price_or_returns_calc='price', include_signal_strength=False, long_only=False)
    in_sample_performance_dict[ticker] = df_long_short_strategy_performance_in_sample
    out_of_sample_performance_dict[ticker] = df_long_short_strategy_performance_out_of_sample

In [ ]:
in_sample_performance_dict['BTC-USD'][in_sample_performance_dict['BTC-USD']['slow_mavg'] == 250]

In [ ]:
in_sample_performance_dict['ETH-USD'][in_sample_performance_dict['ETH-USD']['slow_mavg'] == 250]

In [ ]:
in_sample_performance_dict['LTC-USD'][in_sample_performance_dict['LTC-USD']['slow_mavg'] == 250]

In [ ]:
in_sample_performance_dict['SOL-USD'][in_sample_performance_dict['SOL-USD']['slow_mavg'] == 250]

In [ ]:
in_sample_performance_dict['DOGE-USD'][in_sample_performance_dict['DOGE-USD']['slow_mavg'] == 250]

In [ ]:
df_BTC_in_sample = get_trend_following_strategy_performance(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker='BTC-USD', fast_mavg=50, slow_mavg=250,
                                                            mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, include_transaction_costs_and_fees=True,
                                                            transaction_cost_est=0.005)

In [ ]:
df_BTC_out_of_sample = get_trend_following_strategy_performance(start_date=out_of_sample_start_date, end_date=out_of_sample_end_date, ticker='BTC-USD', fast_mavg=50, slow_mavg=250,
                                                                mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_BTC_full_sample = get_trend_following_strategy_performance(start_date=full_sample_start_date, end_date=full_sample_end_date, ticker='BTC-USD', fast_mavg=50, slow_mavg=250,
                                                                mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_ETH_in_sample = get_trend_following_strategy_performance(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker='ETH-USD', fast_mavg=50, slow_mavg=250,
                                                            mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_ETH_out_of_sample = get_trend_following_strategy_performance(start_date=out_of_sample_start_date, end_date=out_of_sample_end_date, ticker='ETH-USD', fast_mavg=50, slow_mavg=250,
                                                                mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_ETH_full_sample = get_trend_following_strategy_performance(start_date=full_sample_start_date, end_date=full_sample_end_date, ticker='ETH-USD', fast_mavg=50, slow_mavg=250,
                                                                mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_LTC_in_sample = get_trend_following_strategy_performance(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker='LTC-USD', fast_mavg=50, slow_mavg=250,
                                                            mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_LTC_out_of_sample = get_trend_following_strategy_performance(start_date=out_of_sample_start_date, end_date=out_of_sample_end_date, ticker='LTC-USD', fast_mavg=50, slow_mavg=250,
                                                                mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_LTC_full_sample = get_trend_following_strategy_performance(start_date=full_sample_start_date, end_date=full_sample_end_date, ticker='LTC-USD', fast_mavg=50, slow_mavg=250,
                                                                mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_SOL_in_sample = get_trend_following_strategy_performance(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker='SOL-USD', fast_mavg=50, slow_mavg=250,
                                                            mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_SOL_out_of_sample = get_trend_following_strategy_performance(start_date=out_of_sample_start_date, end_date=out_of_sample_end_date, ticker='SOL-USD', fast_mavg=50, slow_mavg=250,
                                                                mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_SOL_full_sample = get_trend_following_strategy_performance(start_date=full_sample_start_date, end_date=full_sample_end_date, ticker='SOL-USD', fast_mavg=50, slow_mavg=250,
                                                                mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_DOGE_in_sample = get_trend_following_strategy_performance(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker='DOGE-USD', fast_mavg=50, slow_mavg=250,
                                                            mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_DOGE_out_of_sample = get_trend_following_strategy_performance(start_date=out_of_sample_start_date, end_date=out_of_sample_end_date, ticker='DOGE-USD', fast_mavg=50, slow_mavg=250,
                                                                mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_DOGE_full_sample = get_trend_following_strategy_performance(start_date=full_sample_start_date, end_date=full_sample_end_date, ticker='DOGE-USD', fast_mavg=50, slow_mavg=250,
                                                                mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

## Long Only Strategy Performance

### In Sample & Out of Sample Performance

In [ ]:
ticker_list = ['BTC-USD','ETH-USD','LTC-USD','SOL-USD','DOGE-USD']
in_sample_long_only_performance_dict = {}
out_of_sample_long_only_performance_dict = {}
for ticker in ticker_list:
    ## Price with Donchian Channels and without Signal Strength Signal
    df_long_short_strategy_performance_in_sample = moving_avg_crossover_with_donchian_strategy_performance(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker=ticker,
                                                                                                           price_or_returns_calc='price', include_signal_strength=False, long_only=True)
    df_long_short_strategy_performance_out_of_sample = moving_avg_crossover_with_donchian_strategy_performance(start_date=out_of_sample_start_date, end_date=out_of_sample_end_date, ticker=ticker,
                                                                                                               price_or_returns_calc='price', include_signal_strength=False, long_only=True)
    in_sample_long_only_performance_dict[ticker] = df_long_short_strategy_performance_in_sample
    out_of_sample_long_only_performance_dict[ticker] = df_long_short_strategy_performance_out_of_sample

In [ ]:
in_sample_long_only_performance_dict['BTC-USD'][in_sample_long_only_performance_dict['BTC-USD']['slow_mavg'] == 250]

In [ ]:
in_sample_long_only_performance_dict['ETH-USD'][in_sample_long_only_performance_dict['ETH-USD']['slow_mavg'] == 250]

In [ ]:
in_sample_long_only_performance_dict['LTC-USD'][in_sample_long_only_performance_dict['LTC-USD']['slow_mavg'] == 250]

In [ ]:
in_sample_long_only_performance_dict['SOL-USD'][in_sample_long_only_performance_dict['SOL-USD']['slow_mavg'] == 250]

In [ ]:
in_sample_long_only_performance_dict['DOGE-USD'][in_sample_long_only_performance_dict['DOGE-USD']['slow_mavg'] == 250]

In [ ]:
df_BTC_long_only_in_sample = get_trend_following_strategy_performance(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker='BTC-USD', fast_mavg=50, slow_mavg=250,
                                                            mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, long_only=True, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_BTC_long_only_out_of_sample = get_trend_following_strategy_performance(start_date=out_of_sample_start_date, end_date=out_of_sample_end_date, ticker='BTC-USD', fast_mavg=50, slow_mavg=250,
                                                                          mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, long_only=True, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_BTC_long_only_full_sample = get_trend_following_strategy_performance(start_date=full_sample_start_date, end_date=full_sample_end_date, ticker='BTC-USD', fast_mavg=50, slow_mavg=250,
                                                                        mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, long_only=True, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_ETH_long_only_in_sample = get_trend_following_strategy_performance(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker='ETH-USD', fast_mavg=50, slow_mavg=250,
                                                                      mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, long_only=True, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_ETH_long_only_out_of_sample = get_trend_following_strategy_performance(start_date=out_of_sample_start_date, end_date=out_of_sample_end_date, ticker='ETH-USD', fast_mavg=50, slow_mavg=250,
                                                                          mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, long_only=True, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_ETH_long_only_full_sample = get_trend_following_strategy_performance(start_date=full_sample_start_date, end_date=full_sample_end_date, ticker='ETH-USD', fast_mavg=50, slow_mavg=250,
                                                                        mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, long_only=True, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_LTC_long_only_in_sample = get_trend_following_strategy_performance(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker='LTC-USD', fast_mavg=50, slow_mavg=250,
                                                                      mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, long_only=True, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_LTC_long_only_out_of_sample = get_trend_following_strategy_performance(start_date=out_of_sample_start_date, end_date=out_of_sample_end_date, ticker='LTC-USD', fast_mavg=50, slow_mavg=250,
                                                                          mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, long_only=True, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_LTC_long_only_full_sample = get_trend_following_strategy_performance(start_date=full_sample_start_date, end_date=full_sample_end_date, ticker='LTC-USD', fast_mavg=50, slow_mavg=250,
                                                                        mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, long_only=True, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_SOL_long_only_in_sample = get_trend_following_strategy_performance(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker='SOL-USD', fast_mavg=50, slow_mavg=250,
                                                                      mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, long_only=True, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_SOL_long_only_out_of_sample = get_trend_following_strategy_performance(start_date=out_of_sample_start_date, end_date=out_of_sample_end_date, ticker='SOL-USD', fast_mavg=50, slow_mavg=250,
                                                                          mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, long_only=True, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_SOL_long_only_full_sample = get_trend_following_strategy_performance(start_date=full_sample_start_date, end_date=full_sample_end_date, ticker='SOL-USD', fast_mavg=50, slow_mavg=250,
                                                                        mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, long_only=True, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_DOGE_in_sample = get_trend_following_strategy_performance(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker='DOGE-USD', fast_mavg=50, slow_mavg=250,
                                                            mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_DOGE_out_of_sample = get_trend_following_strategy_performance(start_date=out_of_sample_start_date, end_date=out_of_sample_end_date, ticker='DOGE-USD', fast_mavg=50, slow_mavg=250,
                                                                mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

In [ ]:
df_DOGE_full_sample = get_trend_following_strategy_performance(start_date=full_sample_start_date, end_date=full_sample_end_date, ticker='DOGE-USD', fast_mavg=50, slow_mavg=250,
                                                                mavg_stepsize=6, rolling_donchian_window=20, rolling_sharpe_window=50, include_transaction_costs_and_fees=True, transaction_cost_est=0.005)

## Strategy Performance for Different Macro Factors

In [ ]:
full_sample_market_start_date = full_sample_start_date - pd.Timedelta('2Y')
in_sample_market_start_date = in_sample_start_date - pd.Timedelta('2Y')

In [ ]:
## Pull Macro Data
macro_indicators = {
    'hpi': 'CSUSHPINSA', ## Case Schiller Home Price Index
    'cpi': 'CPIAUCSL', ## Consumer Price Index
    'ppi_fis': 'PPIFIS', ## Producer Price Index by Commodity for Final Demand: Finished Goods
    'ppi_aco': 'PPIACO', ## Producer Price Index by Commodity for All Commodities
    'pce': 'PCE', ## Personal Consumption Expenditure Index
    'mich_inflation_exp': 'MICH', ## Univ. of Michigan Inflation Expectations
    'fed_price_pressure': 'STLPPM', ## St. Louis Fed Price Pressures Measure
    'oecd_cpi': 'OECDCPALTT01GYM', ## OECD Consumer Price Index: All Groups: Total for the OECD Member Countries
    'm2sl': 'M2SL', ## M2 Money Supply
    'us_gdp': 'A191RL1Q225SBEA', ## US GDP
    'fed_funds': 'FEDFUNDS', ## Federal Funds Rate
    'unemployment_rate': 'UNRATE' ## Unemployment Rate
}

# macro_data = {}
macro_data_dfs = []
for name, series_id in macro_indicators.items():
    macro_data = pdr.DataReader(series_id, 'fred', in_sample_market_start_date, (in_sample_end_date + pd.Timedelta('30d')))
    macro_data_dfs.append(macro_data)

# Concatenate DataFrames on the date index
df_macro_data = pd.concat(macro_data_dfs, axis=1)
col_names = list(macro_indicators.keys())
df_macro_data.columns = col_names
df_macro_data['us_gdp'] = df_macro_data['us_gdp'].ffill()
df_macro_data['us_gdp_daily'] = df_macro_data['us_gdp']/90

## Calculate Inflation Metrics
df_macro_data['cpi_inflation_rate'] = df_macro_data['cpi'].pct_change(12).fillna(0) * 100
df_macro_data['ppi_fis_inflation_rate'] = df_macro_data['ppi_fis'].pct_change(12).fillna(0) * 100
df_macro_data['ppi_aco_inflation_rate'] = df_macro_data['ppi_aco'].pct_change(12).fillna(0) * 100
df_macro_data['oecd_cpi_inflation_rate'] = df_macro_data['oecd_cpi'].pct_change(12).fillna(0) * 100
df_macro_data['pce_inflation_rate'] = df_macro_data['pce'].pct_change(12).fillna(0) * 100

## Create Trimmed Dataframe
macro_cols = ['cpi_inflation_rate','ppi_fis_inflation_rate','ppi_aco_inflation_rate',
             'oecd_cpi_inflation_rate','pce_inflation_rate','us_gdp_daily','fed_funds',
             'unemployment_rate','m2sl']
df_macro_data = df_macro_data[macro_cols]
df_macro_data = df_macro_data.resample('D').ffill()

## Pull VIX Index
vix = get_close_prices(start_date=in_sample_market_start_date, end_date=in_sample_end_date, ticker='^VIX')
df_macro_data = (pd.merge(df_macro_data, vix, left_index=True, right_index=True, how='left')
                 .rename(columns={'^VIX':'vix'}))
df_macro_data['vix'] = df_macro_data['vix'].ffill()

In [ ]:
df_macro_data.tail(20)

In [ ]:
fig = plt.figure(figsize=(15,8))
# plt.style.use('bmh')
layout = (1,1)
inflation_ax = plt.subplot2grid(layout, (0,0), colspan=2)

_ = inflation_ax.plot(df_macro_data.index, df_macro_data['cpi_inflation_rate'], label='CPI Inflation')
_ = inflation_ax.plot(df_macro_data.index, df_macro_data['ppi_fis_inflation_rate'], label='PPI FIS Inflation')
_ = inflation_ax.plot(df_macro_data.index, df_macro_data['ppi_aco_inflation_rate'], label='PPI ACO Inflation')
_ = inflation_ax.plot(df_macro_data.index, df_macro_data['pce_inflation_rate'], label='PCE Inflation')
# _ = inflation_ax.plot(df_macro_data.index, df_macro_data['oecd_cpi_inflation_rate'], label='OECD CPI Inflation')
_ = inflation_ax.grid()
_ = inflation_ax.legend()

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(15,8))
# plt.style.use('bmh')
layout = (1,1)
m2sl_ax = plt.subplot2grid(layout, (0,0), colspan=2)

m2sl_ax2 = m2sl_ax.twinx()
_ = m2sl_ax.plot(df_macro_data.index, df_macro_data['m2sl'], label='M2 Money Stock', color='orange')
_ = m2sl_ax2.plot(df_macro_data.index, df_macro_data['us_gdp_daily'], label='US GDP', color='blue')
# plt.title('M2 Money Stock Over Time')
_ = m2sl_ax.set_xlabel('Date')
_ = m2sl_ax.set_ylabel('M2 ($ Billion)')
# plt.ylabel('M2 ($ Billion)')
_ = m2sl_ax.legend()
_ = m2sl_ax2.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
# plt.style.use('bmh')
layout = (1,1)
fed_funds_ax = plt.subplot2grid(layout, (0,0), colspan=2)

fed_funds_ax2 = fed_funds_ax.twinx()
_ = fed_funds_ax.plot(df_macro_data.index, df_macro_data['fed_funds'], label='Fed Funds Rate', color='orange')
_ = fed_funds_ax2.plot(df_macro_data.index, df_macro_data['unemployment_rate'], label='Unemployment Rate', color='blue')
# plt.title('M2 Money Stock Over Time')
_ = fed_funds_ax.set_xlabel('Date')
_ = fed_funds_ax.set_ylabel('Fed Funds Rate')
_ = fed_funds_ax2.set_ylabel('Unemployment Rate')
_ = fed_funds_ax.legend(loc='upper left')
_ = fed_funds_ax2.legend(loc='upper right')

plt.tight_layout()

In [ ]:
fast_mavg = 50
slow_mavg = 250
mavg_stepsize = 6
rolling_donchian_window = 20
ticker_list = ['BTC-USD','ETH-USD','BNB-USD','SOL-USD']
df_returns = pd.DataFrame()
for ticker in ticker_list:
    df_trend = generate_trend_signal_with_donchian_channel(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker=ticker, fast_mavg=fast_mavg, slow_mavg=slow_mavg,
                                                           mavg_stepsize=mavg_stepsize, price_or_returns_calc='price', rolling_donchian_window=20, include_signal_strength=False)
    strategy_returns_col_name = [f'{ticker}_pct_returns', f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns']
    df_returns[strategy_returns_col_name] = df_trend[strategy_returns_col_name]

df_macro_data = df_macro_data.shift(1)
df_returns = pd.merge(df_returns, df_macro_data, left_index=True, right_index=True, how='left')

In [ ]:
for ticker in ticker_list:
    null_cond = (df_returns[f'{ticker}_pct_returns'].notnull())
    df_temp = df_returns[null_cond]
    macro_cols = df_macro_data.columns.tolist()
    macro_cols.extend([f'{ticker}_pct_returns', f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'])
    fig = plt.figure(figsize=(30,12))
    # plt.style.use('bmh')
    layout = (1,3)
    corr_ax = plt.subplot2grid(layout, (0,0), colspan=2)

    _ = sns.heatmap(df_temp[macro_cols].corr(method='pearson'), annot=True, fmt=".2f", cmap='RdYlGn', linewidths=.5, ax=corr_ax)
    _ = corr_ax.set_title(f'Macro Factor Correlation Heatmap {ticker}')
#     _ = corr_ax.set_ylabel('Slow Moving Average (Days)')
#     _ = corr_ax.set_xlabel('Fast Moving Average (Days)')

    plt.tight_layout()

In [ ]:
## Rolling Correlations
rolling_corr_window = 126
for ticker in ticker_list:
    null_cond = (df_returns[f'{ticker}_pct_returns'].notnull())
    df_temp = df_returns[null_cond]
    rolling_corr_cols = ['cpi_inflation_rate','us_gdp_daily','fed_funds']#,'unemployment_rate','m2sl','vix']
    strategy_return_col = f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'
#     rolling_corr_cols.extend([f'{ticker}_pct_returns', strategy_return_col])
    
    fig = plt.figure(figsize=(30,15))
    layout = (2,2)
    rolling_strategy_corr_ax = plt.subplot2grid(layout, (0,0), colspan=2)
    rolling_ticker_corr_ax = plt.subplot2grid(layout, (1,0), colspan=2)
    for col in rolling_corr_cols:
        df_temp[f'{ticker}_strategy_return_{col}_rolling_{rolling_corr_window}_corr'] = df_temp[strategy_return_col].rolling(rolling_corr_window).corr(df_temp[col])
        df_temp[f'{ticker}_pct_return_{col}_rolling_{rolling_corr_window}_corr'] = df_temp[f'{ticker}_pct_returns'].rolling(rolling_corr_window).corr(df_temp[col])
        
        _ = rolling_strategy_corr_ax.plot(df_temp.index, df_temp[f'{ticker}_strategy_return_{col}_rolling_{rolling_corr_window}_corr'], label=col)
        _ = rolling_ticker_corr_ax.plot(df_temp.index, df_temp[f'{ticker}_pct_return_{col}_rolling_{rolling_corr_window}_corr'], label=col)
    _ = rolling_strategy_corr_ax.legend()
    _ = rolling_strategy_corr_ax.set_title(f'{ticker} Trend Following Strategy Rolling Correlation')
    _ = rolling_ticker_corr_ax.legend()
    _ = rolling_ticker_corr_ax.set_title(f'{ticker} Rolling Correlation')
    plt.tight_layout()

In [ ]:
## Rolling Correlations
rolling_corr_window = 126
for ticker in ticker_list:
    null_cond = (df_returns[f'{ticker}_pct_returns'].notnull())
    df_temp = df_returns[null_cond]
    rolling_corr_cols = ['unemployment_rate','m2sl','vix']
    strategy_return_col = f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'
#     rolling_corr_cols.extend([f'{ticker}_pct_returns', strategy_return_col])
    
    fig = plt.figure(figsize=(30,15))
    layout = (2,2)
    rolling_strategy_corr_ax = plt.subplot2grid(layout, (0,0), colspan=2)
    rolling_ticker_corr_ax = plt.subplot2grid(layout, (1,0), colspan=2)
    for col in rolling_corr_cols:
        df_temp[f'{ticker}_strategy_return_{col}_rolling_{rolling_corr_window}_corr'] = df_temp[strategy_return_col].rolling(rolling_corr_window).corr(df_temp[col])
        df_temp[f'{ticker}_pct_return_{col}_rolling_{rolling_corr_window}_corr'] = df_temp[f'{ticker}_pct_returns'].rolling(rolling_corr_window).corr(df_temp[col])
        
        _ = rolling_strategy_corr_ax.plot(df_temp.index, df_temp[f'{ticker}_strategy_return_{col}_rolling_{rolling_corr_window}_corr'], label=col)
        _ = rolling_ticker_corr_ax.plot(df_temp.index, df_temp[f'{ticker}_pct_return_{col}_rolling_{rolling_corr_window}_corr'], label=col)
    _ = rolling_strategy_corr_ax.legend()
    _ = rolling_strategy_corr_ax.set_title(f'{ticker} Trend Following Strategy Rolling Correlation')
    _ = rolling_ticker_corr_ax.legend()
    _ = rolling_ticker_corr_ax.set_title(f'{ticker} Rolling Correlation')
    plt.tight_layout()

## Strategy Performance Correlation with Different Stock Indices

In [ ]:
## Pull Stock Market Indices
indices = {
    'S&P 500': '^GSPC',
    'NASDAQ Composite': '^IXIC',
    'Dow Jones Industrial Average': '^DJI',
    'FTSE 100': '^FTSE',
    'DAX 30': '^GDAXI',
    'Nikkei 225': '^N225',
    'Hang Seng': '^HSI',
    'Russell 2000': '^RUT',
    'MSCI World': 'URTH',  # Using iShares MSCI World ETF as a proxy
    'BSE Sensex': '^BSESN'
}

stock_index_list = ['^GSPC','^IXIC','^DJI','^FTSE','^GDAXI','^N225','^HSI','^RUT','URTH','^BSESN']
df_stock_index = get_close_prices(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker=stock_index_list)

for stock in stock_index_list:
    df_stock_index[f'{stock}_pct_return'] = df_stock_index[f'{stock}'].pct_change()

In [ ]:
fast_mavg = 50
slow_mavg = 250
mavg_stepsize = 6
rolling_donchian_window = 20
ticker_list = ['BTC-USD','ETH-USD','BNB-USD','SOL-USD']
df_returns = pd.DataFrame()
for ticker in ticker_list:
    df_trend = generate_trend_signal_with_donchian_channel(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker=ticker, fast_mavg=fast_mavg, slow_mavg=slow_mavg,
                                                           mavg_stepsize=mavg_stepsize, price_or_returns_calc='price', rolling_donchian_window=20, include_signal_strength=False)
    strategy_returns_col_name = [f'{ticker}_pct_returns', f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns']
    df_returns[strategy_returns_col_name] = df_trend[strategy_returns_col_name]

return_cols = [f'{stock}_pct_return' for stock in stock_index_list]
df_stock_returns = df_stock_index[return_cols]
df_stock_returns = df_stock_returns.reindex(df_returns.index).ffill()

# df_macro_data = df_macro_data.shift(1)
df_returns = pd.merge(df_returns, df_stock_returns, left_index=True, right_index=True, how='left')

In [ ]:
for ticker in ticker_list:
    null_cond = (df_returns[f'{ticker}_pct_returns'].notnull()) & (df_returns.index.get_level_values('Date') >= '2014-09-29')
    df_temp = df_returns[null_cond]
#     stock_index_cols = []
    stock_index_cols = [f'{stock}_pct_return' for stock in stock_index_list]
    stock_index_cols.extend([f'{ticker}_pct_returns', f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'])
    
    fig = plt.figure(figsize=(30,12))
    # plt.style.use('bmh')
    layout = (1,3)
    corr_ax = plt.subplot2grid(layout, (0,0), colspan=2)

    _ = sns.heatmap(df_temp[stock_index_cols].corr(method='pearson'), annot=True, fmt=".2f", cmap='RdYlGn', linewidths=.5, ax=corr_ax)
    _ = corr_ax.set_title(f'Stock Index Correlation Heatmap {ticker}')

    plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(22,10))
layout = (1,2)
rolling_strategy_corr_ax = plt.subplot2grid(layout, (0,0), colspan=2)

for stock in stock_index_list:
    _ = rolling_strategy_corr_ax.plot(df_stock_returns.index, df_stock_returns[f'{stock}_pct_return'].rolling(100).mean(), label=stock)

_ = rolling_strategy_corr_ax.grid()
_ = rolling_strategy_corr_ax.legend()
_ = rolling_strategy_corr_ax.set_title('Stock Index Returns')

plt.tight_layout()

In [ ]:
## Rolling Correlations
rolling_corr_window = 50
for ticker in ticker_list:
    null_cond = (df_returns[f'{ticker}_pct_returns'].notnull())
    df_temp = df_returns[null_cond]
    stock_index_cols = [f'{stock}_pct_return' for stock in stock_index_list]
    strategy_return_col = f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'
#     rolling_corr_cols.extend([f'{ticker}_pct_returns', strategy_return_col])
    
    fig = plt.figure(figsize=(30,10))
    layout = (2,2)
    rolling_strategy_corr_ax = plt.subplot2grid(layout, (0,0), colspan=2)
    rolling_ticker_corr_ax = plt.subplot2grid(layout, (1,0), colspan=2)
    for col in stock_index_cols[0:5]:
        df_temp[f'{ticker}_strategy_return_{col}_rolling_{rolling_corr_window}_corr'] = df_temp[strategy_return_col].rolling(rolling_corr_window).corr(df_temp[col])
        df_temp[f'{ticker}_pct_return_{col}_rolling_{rolling_corr_window}_corr'] = df_temp[f'{ticker}_pct_returns'].rolling(rolling_corr_window).corr(df_temp[col])
        
        _ = rolling_strategy_corr_ax.plot(df_temp.index, df_temp[f'{ticker}_strategy_return_{col}_rolling_{rolling_corr_window}_corr'], label=col)
        _ = rolling_ticker_corr_ax.plot(df_temp.index, df_temp[f'{ticker}_pct_return_{col}_rolling_{rolling_corr_window}_corr'], label=col)
    _ = rolling_strategy_corr_ax.set_ylim(bottom=-1, top=1)
    _ = rolling_strategy_corr_ax.legend()
    _ = rolling_strategy_corr_ax.set_title(f'{ticker} Trend Following Strategy Rolling Correlation')
    _ = rolling_ticker_corr_ax.set_ylim(bottom=-1, top=1)
    _ = rolling_ticker_corr_ax.legend()
    _ = rolling_ticker_corr_ax.set_title(f'{ticker} Rolling Correlation')
    _ = rolling_strategy_corr_ax.grid()
    _ = rolling_ticker_corr_ax.grid()
    
    plt.tight_layout()

In [ ]:
## Rolling Correlations
rolling_corr_window = 50
for ticker in ticker_list:
    null_cond = (df_returns[f'{ticker}_pct_returns'].notnull())
    df_temp = df_returns[null_cond]
    stock_index_cols = [f'{stock}_pct_return' for stock in stock_index_list]
    strategy_return_col = f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'
#     rolling_corr_cols.extend([f'{ticker}_pct_returns', strategy_return_col])
    
    fig = plt.figure(figsize=(30,10))
    layout = (2,2)
    rolling_strategy_corr_ax = plt.subplot2grid(layout, (0,0), colspan=2)
    rolling_ticker_corr_ax = plt.subplot2grid(layout, (1,0), colspan=2)
    for col in stock_index_cols[5:10]:
        df_temp[f'{ticker}_strategy_return_{col}_rolling_{rolling_corr_window}_corr'] = df_temp[strategy_return_col].rolling(rolling_corr_window).corr(df_temp[col])
        df_temp[f'{ticker}_pct_return_{col}_rolling_{rolling_corr_window}_corr'] = df_temp[f'{ticker}_pct_returns'].rolling(rolling_corr_window).corr(df_temp[col])
        
        _ = rolling_strategy_corr_ax.plot(df_temp.index, df_temp[f'{ticker}_strategy_return_{col}_rolling_{rolling_corr_window}_corr'], label=col)
        _ = rolling_ticker_corr_ax.plot(df_temp.index, df_temp[f'{ticker}_pct_return_{col}_rolling_{rolling_corr_window}_corr'], label=col)
    _ = rolling_strategy_corr_ax.set_ylim(bottom=-1, top=1)
    _ = rolling_strategy_corr_ax.legend()
    _ = rolling_strategy_corr_ax.set_title(f'{ticker} Trend Following Strategy Rolling Correlation')
    _ = rolling_ticker_corr_ax.set_ylim(bottom=-1, top=1)
    _ = rolling_ticker_corr_ax.legend()
    _ = rolling_ticker_corr_ax.set_title(f'{ticker} Rolling Correlation')
    _ = rolling_strategy_corr_ax.grid()
    _ = rolling_ticker_corr_ax.grid()
    
    plt.tight_layout()

## Strategy Performance Correlation with Standard Deviation of Crpyto Prices

In [ ]:
ticker_list = ['BTC-USD','ETH-USD','BNB-USD','SOL-USD']
df_crypto_prices_std = get_close_prices(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker=ticker_list)
df_crypto_returns_std = get_close_prices(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker=ticker_list)

for ticker in ticker_list:
#     df_crypto_prices_std[f'{ticker}_20d_price_std'] = df_crypto_prices_std[ticker].rolling(20).std()
    df_crypto_returns_std = get_returns_volatility(df_crypto_returns_std, vol_range_list=[5, 10, 20], close_px_col=ticker)

In [ ]:
fast_mavg = 50
slow_mavg = 250
mavg_stepsize = 6
rolling_donchian_window = 20
ticker_list = ['BTC-USD','ETH-USD','BNB-USD','SOL-USD']
df_returns = pd.DataFrame()
for ticker in ticker_list:
    df_trend = generate_trend_signal_with_donchian_channel(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker=ticker, fast_mavg=fast_mavg, slow_mavg=slow_mavg,
                                                           mavg_stepsize=mavg_stepsize, price_or_returns_calc='price', rolling_donchian_window=20, include_signal_strength=False)
    strategy_returns_col_name = [f'{ticker}', f'{ticker}_pct_returns', f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns']
    df_returns[strategy_returns_col_name] = df_trend[strategy_returns_col_name]

# price_std_cols = [f'{ticker}_20d_price_std' for ticker in ticker_list]
returns_std_cols = [f'{ticker}_volatility_5' for ticker in ticker_list] + [f'{ticker}_volatility_10' for ticker in ticker_list] + [f'{ticker}_volatility_20' for ticker in ticker_list]
# df_prices_std = pd.merge(df_returns, df_crypto_prices_std[price_std_cols], left_index=True, right_index=True, how='left')

df_volatility = pd.merge(df_returns, df_crypto_returns_std[returns_std_cols], left_index=True, right_index=True, how='left')

In [ ]:
ticker='BTC-USD'
null_cond = (df_volatility[f'{ticker}_pct_returns'].notnull()) & (df_volatility.index.get_level_values('Date') >= '2014-09-29')
df_volatility[null_cond][std_cols].head(20)

In [ ]:
print(df_volatility[null_cond]['BTC-USD_volatility_5'].corr(df_volatility[null_cond]['BTC-USD']))
print(df_volatility[null_cond]['BTC-USD_volatility_5'].corr(df_volatility[null_cond]['BTC-USD_pct_returns']))

In [ ]:
for ticker in ticker_list:
    null_cond = (df_volatility[f'{ticker}_pct_returns'].notnull()) & (df_volatility.index.get_level_values('Date') >= '2014-09-29')
    df_temp = df_volatility[null_cond]
#     stock_index_cols = []
    std_cols = [f'{ticker}_volatility_5' for ticker in ticker_list]
    std_cols.extend([f'{ticker}', f'{ticker}_pct_returns', f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'])
    
    fig = plt.figure(figsize=(30,12))
    # plt.style.use('bmh')
    layout = (1,3)
    corr_ax = plt.subplot2grid(layout, (0,0), colspan=2)

    _ = sns.heatmap(df_temp[std_cols].corr(method='pearson'), annot=True, fmt=".2f", cmap='RdYlGn', linewidths=.5, ax=corr_ax)
    _ = corr_ax.set_title(f'Crypto Returns 5 Day Volatility Correlation Heatmap {ticker}')

    plt.tight_layout()

In [ ]:
for ticker in ticker_list:
    null_cond = (df_volatility[f'{ticker}_pct_returns'].notnull()) & (df_volatility.index.get_level_values('Date') >= '2014-09-29')
    df_temp = df_volatility[null_cond]
#     stock_index_cols = []
    std_cols = [f'{ticker}_volatility_20' for ticker in ticker_list]
    std_cols.extend([f'{ticker}', f'{ticker}_pct_returns', f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'])
    
    fig = plt.figure(figsize=(30,12))
    # plt.style.use('bmh')
    layout = (1,3)
    corr_ax = plt.subplot2grid(layout, (0,0), colspan=2)

    _ = sns.heatmap(df_temp[std_cols].corr(method='pearson'), annot=True, fmt=".2f", cmap='RdYlGn', linewidths=.5, ax=corr_ax)
    _ = corr_ax.set_title(f'Crypto Returns 20 Day Volatility Correlation Heatmap {ticker}')

    plt.tight_layout()

In [ ]:
for ticker in ticker_list:
    null_cond = (df_volatility[f'{ticker}_pct_returns'].notnull()) & (df_volatility.index.get_level_values('Date') >= '2014-09-29')
    df_temp = df_volatility[null_cond]
#     stock_index_cols = []
    std_cols = [f'{ticker}_volatility_20' for ticker in ticker_list]
    std_cols.extend([f'{ticker}', f'{ticker}_pct_returns', f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'])
    
    fig = plt.figure(figsize=(30,12))
    # plt.style.use('bmh')
    layout = (1,3)
    corr_ax = plt.subplot2grid(layout, (0,0), colspan=2)

    _ = sns.heatmap(df_temp[std_cols].corr(method='pearson'), annot=True, fmt=".2f", cmap='RdYlGn', linewidths=.5, ax=corr_ax)
    _ = corr_ax.set_title(f'Crypto Returns 20 Day Volatility Correlation Heatmap {ticker}')

    plt.tight_layout()

In [ ]:
df_SP500 = get_close_prices(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker='^GSPC')
df_SP500 = get_returns_volatility(df_SP500, vol_range_list=[20], close_px_col='^GSPC')

In [ ]:
fig = plt.figure(figsize=(22,10))
layout = (1,2)
rolling_strategy_corr_ax = plt.subplot2grid(layout, (0,0), colspan=2)

for ticker in ticker_list:
    _ = rolling_strategy_corr_ax.plot(df_crypto_returns_std.index, df_crypto_returns_std[f'{ticker}_volatility_20'] * np.sqrt(365), label=ticker)

_ = rolling_strategy_corr_ax.plot(df_SP500.index, df_SP500[f'^GSPC_volatility_20'] * np.sqrt(252), label='S&P 500')
_ = rolling_strategy_corr_ax.grid()
_ = rolling_strategy_corr_ax.legend()
_ = rolling_strategy_corr_ax.set_title('Crypto Annualized Volatility')

plt.tight_layout()

In [ ]:
## Rolling Correlations
rolling_corr_window = 50
for ticker in ticker_list:
#     null_cond = (df_returns[f'{ticker}_pct_returns'].notnull())
    null_cond = (df_volatility[f'{ticker}_pct_returns'].notnull()) & (df_volatility.index.get_level_values('Date') >= '2014-09-29')
    df_temp = df_volatility[null_cond]
    volatility_cols = [f'{crypto}_volatility_5' for crypto in ticker_list]
    strategy_return_col = f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'
#     rolling_corr_cols.extend([f'{ticker}_pct_returns', strategy_return_col])
    
    fig = plt.figure(figsize=(30,10))
    layout = (2,2)
    rolling_strategy_corr_ax = plt.subplot2grid(layout, (0,0), colspan=2)
    rolling_ticker_corr_ax = plt.subplot2grid(layout, (1,0), colspan=2)
    for col in volatility_cols:
        df_temp[f'{ticker}_strategy_return_{col}_rolling_{rolling_corr_window}_corr'] = df_temp[strategy_return_col].rolling(rolling_corr_window).corr(df_temp[col])
        df_temp[f'{ticker}_pct_return_{col}_rolling_{rolling_corr_window}_corr'] = df_temp[f'{ticker}_pct_returns'].rolling(rolling_corr_window).corr(df_temp[col])
        
        _ = rolling_strategy_corr_ax.plot(df_temp.index, df_temp[f'{ticker}_strategy_return_{col}_rolling_{rolling_corr_window}_corr'], label=col)
        _ = rolling_ticker_corr_ax.plot(df_temp.index, df_temp[f'{ticker}_pct_return_{col}_rolling_{rolling_corr_window}_corr'], label=col)
    _ = rolling_strategy_corr_ax.set_ylim(bottom=-1, top=1)
    _ = rolling_strategy_corr_ax.legend()
    _ = rolling_strategy_corr_ax.set_title(f'{ticker} Trend Following Strategy Rolling Correlation with 5 Day Volatility')
    _ = rolling_ticker_corr_ax.set_ylim(bottom=-1, top=1)
    _ = rolling_ticker_corr_ax.legend()
    _ = rolling_ticker_corr_ax.set_title(f'{ticker} Returns Rolling Correlation')
    _ = rolling_strategy_corr_ax.grid()
    _ = rolling_ticker_corr_ax.grid()
    
    plt.tight_layout()

In [ ]:
## Rolling Correlations
rolling_corr_window = 50
for ticker in ticker_list:
#     null_cond = (df_returns[f'{ticker}_pct_returns'].notnull())
    null_cond = (df_volatility[f'{ticker}_pct_returns'].notnull()) & (df_volatility.index.get_level_values('Date') >= '2014-09-29')
    df_temp = df_volatility[null_cond]
    volatility_cols = [f'{crypto}_volatility_10' for crypto in ticker_list]
    strategy_return_col = f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'
#     rolling_corr_cols.extend([f'{ticker}_pct_returns', strategy_return_col])
    
    fig = plt.figure(figsize=(30,10))
    layout = (2,2)
    rolling_strategy_corr_ax = plt.subplot2grid(layout, (0,0), colspan=2)
    rolling_ticker_corr_ax = plt.subplot2grid(layout, (1,0), colspan=2)
    for col in volatility_cols:
        df_temp[f'{ticker}_strategy_return_{col}_rolling_{rolling_corr_window}_corr'] = df_temp[strategy_return_col].rolling(rolling_corr_window).corr(df_temp[col])
        df_temp[f'{ticker}_pct_return_{col}_rolling_{rolling_corr_window}_corr'] = df_temp[f'{ticker}_pct_returns'].rolling(rolling_corr_window).corr(df_temp[col])
        
        _ = rolling_strategy_corr_ax.plot(df_temp.index, df_temp[f'{ticker}_strategy_return_{col}_rolling_{rolling_corr_window}_corr'], label=col)
        _ = rolling_ticker_corr_ax.plot(df_temp.index, df_temp[f'{ticker}_pct_return_{col}_rolling_{rolling_corr_window}_corr'], label=col)
    _ = rolling_strategy_corr_ax.set_ylim(bottom=-1, top=1)
    _ = rolling_strategy_corr_ax.legend()
    _ = rolling_strategy_corr_ax.set_title(f'{ticker} Trend Following Strategy Rolling Correlation with 10 Day Volatility')
    _ = rolling_ticker_corr_ax.set_ylim(bottom=-1, top=1)
    _ = rolling_ticker_corr_ax.legend()
    _ = rolling_ticker_corr_ax.set_title(f'{ticker} Returns Rolling Correlation')
    _ = rolling_strategy_corr_ax.grid()
    _ = rolling_ticker_corr_ax.grid()
    
    plt.tight_layout()

## Strategy Performance Correlation with Crypto Trading Volume

In [ ]:
ticker_list = ['BTC-USD','ETH-USD','BNB-USD','SOL-USD']
df_crypto_volume = load_financial_data(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker=ticker_list)
df_crypto_volume = df_crypto_volume['Volume']
df_crypto_volume.columns = [f'{ticker}_volume' for ticker in ticker_list]

In [ ]:
df_crypto_volume.head()

In [ ]:
fast_mavg = 50
slow_mavg = 250
mavg_stepsize = 6
rolling_donchian_window = 20
ticker_list = ['BTC-USD','ETH-USD','BNB-USD','SOL-USD']
df_returns = pd.DataFrame()
for ticker in ticker_list:
    df_trend = generate_trend_signal_with_donchian_channel(start_date=in_sample_start_date, end_date=in_sample_end_date, ticker=ticker, fast_mavg=fast_mavg, slow_mavg=slow_mavg,
                                                           mavg_stepsize=mavg_stepsize, price_or_returns_calc='price', rolling_donchian_window=20, include_signal_strength=False)
    strategy_returns_col_name = [f'{ticker}', f'{ticker}_pct_returns', f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns']
    df_returns[strategy_returns_col_name] = df_trend[strategy_returns_col_name]

volume_cols = [f'{ticker}_volume' for ticker in ticker_list]
df_volume = pd.merge(df_returns, df_crypto_volume[volume_cols], left_index=True, right_index=True, how='left')

In [ ]:
print(df_volatility[null_cond]['BTC-USD_volatility_5'].corr(df_volatility[null_cond]['BTC-USD']))
print(df_volatility[null_cond]['BTC-USD_volatility_5'].corr(df_volatility[null_cond]['BTC-USD_pct_returns']))

In [ ]:
for ticker in ticker_list:
    null_cond = (df_volume[f'{ticker}_pct_returns'].notnull()) & (df_volume.index.get_level_values('Date') >= '2014-09-29')
    df_temp = df_volume[null_cond]
#     stock_index_cols = []
    volume_cols = [f'{ticker}_volume' for ticker in ticker_list]
    volume_cols.extend([f'{ticker}', f'{ticker}_pct_returns', f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'])
    
    fig = plt.figure(figsize=(30,12))
    # plt.style.use('bmh')
    layout = (1,3)
    corr_ax = plt.subplot2grid(layout, (0,0), colspan=2)

    _ = sns.heatmap(df_temp[volume_cols].corr(method='pearson'), annot=True, fmt=".2f", cmap='RdYlGn', linewidths=.5, ax=corr_ax)
    _ = corr_ax.set_title(f'{ticker} Returns Correlation Heatmap with Trading Volume')

    plt.tight_layout()

In [ ]:
import matplotlib.ticker as ticker

# Define a function to format y-axis in millions
def millions(x, pos):
    return '%1.1fM' % (x * 1e-6)

def billions(x, pos):
    return '%1.1fB' % (x * 1e-9)

fig = plt.figure(figsize=(18,8))
layout = (1,2)
volume_ax = plt.subplot2grid(layout, (0,0), colspan=2)

for ticker in ticker_list:
    _ = volume_ax.plot(df_volume.index, df_volume[f'{ticker}_volume'] * np.sqrt(365), label=ticker)

_ = volume_ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(billions))
# _ = rolling_strategy_corr_ax.plot(df_SP500.index, df_SP500[f'^GSPC_volatility_20'] * np.sqrt(252), label='S&P 500')
_ = volume_ax.grid()
_ = volume_ax.legend()
_ = volume_ax.set_title('Crypto Volume')

plt.tight_layout()

In [ ]:
## Rolling Correlations
rolling_corr_window = 50
for ticker in ticker_list:
#     null_cond = (df_returns[f'{ticker}_pct_returns'].notnull())
    null_cond = (df_volume[f'{ticker}_pct_returns'].notnull()) & (df_volume.index.get_level_values('Date') >= '2014-09-29')
    df_temp = df_volume[null_cond]
    volume_cols = [f'{ticker}_volume' for ticker in ticker_list]
    strategy_return_col = f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'
    
    fig = plt.figure(figsize=(30,10))
    layout = (2,2)
    rolling_strategy_corr_ax = plt.subplot2grid(layout, (0,0), colspan=2)
    rolling_ticker_corr_ax = plt.subplot2grid(layout, (1,0), colspan=2)
    for col in volume_cols:
        df_temp[f'{ticker}_strategy_return_{col}_rolling_{rolling_corr_window}_corr'] = df_temp[strategy_return_col].rolling(rolling_corr_window).corr(df_temp[col])
        df_temp[f'{ticker}_pct_return_{col}_rolling_{rolling_corr_window}_corr'] = df_temp[f'{ticker}_pct_returns'].rolling(rolling_corr_window).corr(df_temp[col])
        
        _ = rolling_strategy_corr_ax.plot(df_temp.index, df_temp[f'{ticker}_strategy_return_{col}_rolling_{rolling_corr_window}_corr'], label=col)
        _ = rolling_ticker_corr_ax.plot(df_temp.index, df_temp[f'{ticker}_pct_return_{col}_rolling_{rolling_corr_window}_corr'], label=col)
    _ = rolling_strategy_corr_ax.set_ylim(bottom=-1, top=1)
    _ = rolling_strategy_corr_ax.legend()
    _ = rolling_strategy_corr_ax.set_title(f'{ticker} Trend Following Strategy Rolling Correlation with Trading Volume')
    _ = rolling_ticker_corr_ax.set_ylim(bottom=-1, top=1)
    _ = rolling_ticker_corr_ax.legend()
    _ = rolling_ticker_corr_ax.set_title(f'{ticker} Returns Rolling Correlation')
    _ = rolling_strategy_corr_ax.grid()
    _ = rolling_ticker_corr_ax.grid()
    
    plt.tight_layout()

In [ ]:
df_trend.head()

In [ ]:
df_crypto_returns_std.head()

In [ ]:
## Rolling Correlations
rolling_corr_window = 50
for ticker in ticker_list:
    null_cond = (df_returns[f'{ticker}_pct_returns'].notnull())
    df_temp = df_returns[null_cond]
    stock_index_cols = [f'{stock}_pct_return' for stock in stock_index_list]
    strategy_return_col = f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'
#     rolling_corr_cols.extend([f'{ticker}_pct_returns', strategy_return_col])
    
    fig = plt.figure(figsize=(30,15))
    layout = (2,2)
    rolling_strategy_corr_ax = plt.subplot2grid(layout, (0,0), colspan=2)
    rolling_ticker_corr_ax = plt.subplot2grid(layout, (1,0), colspan=2)
    for col in stock_index_cols:
        df_temp[f'{ticker}_strategy_return_{col}_rolling_{rolling_corr_window}_corr'] = df_temp[strategy_return_col].rolling(rolling_corr_window).corr(df_temp[col])
        df_temp[f'{ticker}_pct_return_{col}_rolling_{rolling_corr_window}_corr'] = df_temp[f'{ticker}_pct_returns'].rolling(rolling_corr_window).corr(df_temp[col])
        
        _ = rolling_strategy_corr_ax.plot(df_temp.index, df_temp[f'{ticker}_strategy_return_{col}_rolling_{rolling_corr_window}_corr'], label=col)
        _ = rolling_ticker_corr_ax.plot(df_temp.index, df_temp[f'{ticker}_pct_return_{col}_rolling_{rolling_corr_window}_corr'], label=col)
    _ = rolling_strategy_corr_ax.legend()
    _ = rolling_strategy_corr_ax.set_title(f'{ticker} Trend Following Strategy Rolling Correlation')
    _ = rolling_ticker_corr_ax.legend()
    _ = rolling_ticker_corr_ax.set_title(f'{ticker} Rolling Correlation')
    _ = rolling_strategy_corr_ax.grid()
    _ = rolling_ticker_corr_ax.grid()
    
    plt.tight_layout()

In [ ]:
df_returns.head(20)

In [ ]:
df_stock_index.head()

In [ ]:
df_temp.tail(20)

In [ ]:
df_macro_data.tail(20)

In [ ]:
df_returns[macro_cols].corr()

In [ ]:
macro_cols

In [ ]:
df_returns.head(1000)

In [ ]:
df_returns.head(1000)

In [ ]:
df_macro_data.head()